In [1]:
import random
import time
from functools import wraps
  
def fn_timer(function):
  @wraps(function)
  def function_timer(*args, **kwargs):
    t0 = time.process_time()
    result = function(*args, **kwargs)
    t1 = time.process_time()
    print ("Total time running %s: %s s" %
        (function.__name__, str(t1-t0))
        )
    return result
  return function_timer

- 概况  
  1. 通常是实际应用排序中最好的选择，因为其平均性能非常好
    - 期望的时间复杂度为 $\Theta(nlg(n))$，而且 $\Theta(nlg(n))$ 中隐含的常数因子非常小
  2. 能够进行原址排序，可以在虚拟环境中很好的工作
    - 如果输入数组中仅有**常数个元素**需要在排序过程中存储在数组之外，则称排序算法是**原址**的  

## 7.1 快速排序的描述

- 采用分治思想，包括分解，解决和合并

### 数组划分 -- PARTITION  
- 实现对子数组组 $A[p \cdots r]$ 的原址重排

#### 代码实现

In [2]:
def partition(A, p, r):
  x = A[r]  # 主元
  i = p - 1
  for j in range(p, r):
    if A[j] <= x:
      i += 1
      A[i], A[j] = A[j], A[i]
  A[i+1], A[r] = A[r], A[i+1]
  return i + 1

In [3]:
A = [2, 8, 7, 1, 3, 5, 6, 4]
print(partition(A, 0, len(A)-1))
print(A)

3
[2, 1, 3, 4, 7, 5, 6, 8]


#### 执行过程分析

<img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191215160554.png width=600>

#### 快速排序

In [4]:
def quicksort(A, p, r):
  if p < r:
    q = partition(A, p, r)
    quicksort(A, p, q-1)
    quicksort(A, q+1, r)

In [5]:
A = [2, 8, 7, 1, 3, 5, 6, 4]
quicksort(A, 0, len(A)-1)
print(A)

[1, 2, 3, 4, 5, 6, 7, 8]


### 练习

#### 7.1-2

当 $A[p \cdots r]$ 中的元素均相同时，返回的 $q$ 值为 $r$

In [6]:
A = [1 for i in range(10)]
print(partition(A, 0, 9))

9


可按如下方法修改 PARTITION, 使其返回值为 
$ \lfloor (p+r)/2 \rfloor$

In [7]:
def partition_handle_same_value(A, p, r):
  x = A[r]
  i = p - 1
  for j in range(p, r):
    if A[j] <= x:
      if A[j] == x and (j%2) == (p)%2:
        continue
      i += 1
      A[i], A[j] = A[j], A[i]
  A[i], A[r] = A[r], A[i]
  return i + 1

In [8]:
A = [1 for i in range(11)]
print(partition_handle_same_value(A, 0, 9))
print(partition_handle_same_value(A, 0, 10))

4
5


#### 7.1-4  
以非递增序排序

In [9]:
def partition_reverse(A, p, r):
  x = A[r]
  i = p - 1
  for j in range(p, r):
    if A[j] >= x:
      i += 1
      A[i], A[j] = A[j], A[i]
  A[i+1], A[r] = A[r], A[i+1]
  return i + 1

In [10]:
def quicksort_reverse(A, p, r):
  if p < r:
    q = partition_reverse(A, p, r)
    quicksort_reverse(A, p, q-1)
    quicksort_reverse(A, q+1, r)

In [11]:
A = [2, 8, 7, 1, 3, 5, 6, 4]
quicksort_reverse(A, 0, len(A)-1)
print(A)

[8, 7, 6, 5, 4, 3, 2, 1]


## 7.2 快速排序的性能

### 最坏情况划分

- 当划分产生的两个子问题分别包括 $n-1$ 个元素 和 $0$ 个元素时，即为快速排序的是坏情况（证明在 7.4.1 节）
- 假设算法的每一次递归调用都出现了这种最坏情况，则可得递推式：
$$T\left( n \right) = T\left( {n - 1} \right) + T\left( 0 \right) + \Theta \left( 1 \right) = T\left( {n - 1} \right) + \Theta \left( n \right)$$
结果为 $\Theta(n^2)$
- 当数组已经完全有序时，并且主元选为最后一个元素时，即会出现此种情况

### 最好情况划分

- 最好情况划分，即为最平衡的划分，此时一个子问题的规模为 $T( \lfloor n/2 \rfloor)$, 另一个问题的规模为
$T(\lceil n/2 \rceil - 1)$
- 此时，算法运行时间递推式为
$$ T(n) = 2T(n/2) + \Theta(n) $$
由主定理可得， $T(n) = \Theta(nlg(n))$

### 平衡的划分

- 快速排序的平均运行时间更加接近于其最好情况，而非最坏情况
- 考虑 $9:1$ 的不平衡划分，可得递推式
$$ T(n) = T(9n/10) + T(n/10) + \Theta(n) $$
- 通过递推树法进行分析  
<img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191215170107.png width=600>  
可得其运行时间为 $O(nlg(n))$
- 任何一种常数比例的划分都会产生深度为 $\Theta(lg(n))$ 的递推树，其中每一层的时间代价都是 $O(n)$
  - 因此，只要划分是常数比例的，算法的运行时间总是 $O(nlg(n))$

### 对于平均情况的直接观察

- 快速排序的行为依赖于输入数组中元素的值的相对顺序，而不是某些特定值本身，因此需要假设输入数据的所有排列都是等概率的
- 当好和差的划分交替出现时，快速排序的时间复杂度与全是好的划分时是一样的，仍然是 $O(nlg(n))$。区别是 $O$ 符号
中隐含的**常数因子项**略大一些    
  - 将差的划分产生的代价归并入划分步骤中
  - 示意图  
  <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191215171848.png width=600>

## 7.3 快速排序的随机化版本

- 随机化版本的快速排序常作为大数据输入情况下的排序算法
- 可以采用**随机抽样**的随机化技术来实现随机化版本的快速排序
  - PARTITION 时随机的选择主元

### RANDOMIZED-PARTITION

In [12]:
def randomized_partition(A, p, r):
  i = random.randint(p, r)
  A[i], A[r] = A[r], A[i]
  return partition(A, p, r)

### RANDOMIZED-QUICKSORT

In [13]:
def randomized_quicksort(A, p, r):
  if p < r:
    q = randomized_partition(A, p, r)
    randomized_quicksort(A, p, q-1)
    randomized_quicksort(A, q+1, r)

In [14]:
A = [2, 8, 7, 1, 3, 5, 6, 4]
randomized_quicksort(A, 0, len(A)-1)
print(A)

[1, 2, 3, 4, 5, 6, 7, 8]


## 7.4 快速排序分析

### 7.4.1 最坏情况分析

- 设 $T(n)$ 为最坏情况下 QUICKSORT 所花费的时间，则可得递归式
$$T\left( n \right) = \mathop {\max }\limits_{0 \le q \le n - 1} \left( {T\left( q \right) + T\left( {n - q - 1} \right)} \right) + \Theta \left( n \right)$$
- 通过代入法，可证明 $T(n) = O(n^2) $ 且 $T(n)=\Omega(n^2)$，由此可得 $T(n) = \Theta(n^2)$

### 7.4.2 期望运行时间
- 前提假设：数组中各个元素互异

#### 运行时间和比较操作

**引理7.1**: 当在一个包含 $n$ 个元素的数组中运行 QUICKSORT 时，假设在 PARTITION 的第 4 行中所做的比较次数为 $X$
,则 QUICKSORT 的运行时间为 $O(n+x)$

#### 期望运行时间求解

- 将数组 $A$ 中的各个元素复命名为 $z_1, z_2, \cdots, z_n$，其中 $z_i$ 是数组 $A$ 中第 $i$ 小的元素
- 定义 $Z_{ij} = \{ z_i, z_{i+1}, \cdots, z_j \}$
- 定义指示随机变量$$ X_{ij} = I\{ z_i 与 z_j 进行比较 \} $$
- 总的比较次数  $$X = \sum\limits_{i = 1}^{n - 1} {\sum\limits_{j = i + 1}^n {{X_{ij}}} } $$  
  - 在 QUICKSORT 中，两个元素最多只会比较一次
  -因为一个元素一旦被选为主元，其会与 $[p, r]$ 除自身外的所有的其它元素进行一次比较，之后就不会再参与比较
- 由期望的线性特性和指示器随机变量的特性，可得：
$$E\left( X \right) = \sum\limits_{i = 1}^{n - 1} {\sum\limits_{j = i + 1}^n {E\left( {{X_{ij}}} \right)} }  = \sum\limits_{i = 1}^{n - 1} {\sum\limits_{j = i + 1}^n {\Pr \left\{ {{z_i}与{z_j}} 进行比较 \right\}} } $$
- $z_i$ 会与 $z_j$ 进行比较，当且仅当 $Z_{i,j}$ 中被选为主元的第一个元素是 $z_i$ 或 $z_j$
  - 因为一旦一个满足 $z_i < x < z_j$ 的主元 $x$ 被选中后，$z_i$ 与 $z_j$ 以后就再也不可能会被比较了
  - 由此可得：$$\begin{aligned}
    Pr\{z_i 与 z_j 进行比较\} &= Pr\{ z_i 或 z_j 是集合 Z_{ij} 中选出的第一个主元 \} \\
      &= Pr\{ z_i 是集合 Z_{ij} 中选出的第一个主元\} \\
      &+ Pr\{ z_j 是集合 Z_{ij} 中选出的第一个主元\} \\
      &= {1 \over j - i + 1} + {1 \over j - i + 1} =  {2 \over j - i + 1}
  \end{aligned}$$
    - 主元的选择是随机且独立的
- 由上述分析可得： 
  - $$E(X) = \sum\limits_{i = 1}^{n - 1} {\sum\limits_{j = i + 1}^n {2 \over j - i + 1} }
= \sum\limits_{i = 1}^{n - 1} {\sum\limits_{k = 1}^{n-i} {2 \over k + 1} } < 
\sum\limits_{i = 1}^{n - 1} {\sum\limits_{k = 1}^{n} {2 \over k} } = \sum\limits_{i=1}^{n-1}O(lg(n)) = O(nlg(n))
$$  
    - 调和级数的性质
  - $$ \begin{aligned}
  E(X) &= \sum\limits_{i = 1}^{n - 1} {\sum\limits_{j = i + 1}^n {2 \over j - i + 1} } = 
  \sum\limits_{i = 1}^{n - 1} {\sum\limits_{k = 1}^{n-i} {2 \over k + 1} } \ge 
  \sum\limits_{i = 1}^{n - 1} {\int_1^{n - i + 1} {{2 \over {k + 1}}dk} } = 
  \sum\limits_{i = 1}^{n - 1}{2\ln \left( {k + 1} \right)|_{k = 1}^{n - i + 1}} \\
  &= \sum\limits_{i = 1}^{n - 1}{2ln(n-i+2) + 2ln2} = 
  2\ln \prod\limits_{i = 1}^{n - 1} {(n - i + 2)} + \Theta(n) 
  = 2ln({(n+1)! \over 2}) + \Theta(n) \\
  &= 2ln(n!) + \Theta(n) = {2 \over lg(e)}lg(n!) + \Theta(n) \ge cnlg(n)
  \end{aligned}$$
    - 由第 3 章的斯特林公式可推得 $lg(n!) = \Theta(nlg(n))$
- 综上可得： $E(X) = \Theta(nlg(n))$

### 练习

#### 7.4-5 快速排序中加入插入排序来提升运行速度

- 对于快速排序，当长度小于 $k$ 时直接返回，则其递归树的深度为 $O(lg(n/k))$，运行时间为 $O(nlg(n/k))$
- 对于插入排序，对于特定的元素，其最多向前比较 $k$ 个数，就会得出相应的结果，运行时间为 $O(k)$ ，总的运行时间为 $O(nk)$
- 总的运行时间为 $O(nk + nlg(n/k)) $
- 理论上，应选取 $k$ ，使得运行时间最短。但是由于常数项难以确定，因此实际中，可以给定一个较大规模的输入，然后尝试不同的 $k$ 值

##### 代码实现

In [15]:
def insertion_sort(A):
  for j in range(1, len(A)):
    key = A[j]
    i = j - 1
    while i >= 0 and A[i] > key:
      A[i+1] = A[i]
      i -= 1
    A[i+1] = key

In [16]:
def quicksort_with_insertion_sort(A,k):
  def helper(A, p, r):
    if r - p > k:
      q = partition(A, p, r)
      helper(A, p, q-1)
      helper(A, q+1, r)
  helper(A, 0, len(A)-1)
  insertion_sort(A)

In [17]:
A = [random.randint(1, 100) for i in range(20)]
print(A)
quicksort_with_insertion_sort(A, 5)
print(A)

[55, 66, 90, 60, 4, 73, 5, 25, 24, 46, 56, 79, 78, 44, 8, 83, 33, 39, 60, 45]
[4, 5, 8, 24, 25, 33, 39, 44, 45, 46, 55, 56, 60, 60, 66, 73, 78, 79, 83, 90]


In [18]:
A = [random.randint(1, 10000) for i in range(100000)]
B = A.copy()
fn_timer(quicksort)(A, 0, len(A)-1)
for k in range(0, 100, 10):
  A =  B.copy()
  print('k = {}'.format(k), end=': ')
  fn_timer(quicksort_with_insertion_sort)(A, k)


Total time running quicksort: 1.203125 s
k = 0: Total time running quicksort_with_insertion_sort: 1.015625 s
k = 10: Total time running quicksort_with_insertion_sort: 1.03125 s
k = 20: Total time running quicksort_with_insertion_sort: 0.71875 s
k = 30: Total time running quicksort_with_insertion_sort: 0.9375 s
k = 40: Total time running quicksort_with_insertion_sort: 0.921875 s
k = 50: Total time running quicksort_with_insertion_sort: 0.96875 s
k = 60: Total time running quicksort_with_insertion_sort: 1.1875 s
k = 70: Total time running quicksort_with_insertion_sort: 1.1875 s
k = 80: Total time running quicksort_with_insertion_sort: 1.28125 s
k = 90: Total time running quicksort_with_insertion_sort: 1.09375 s


## 思考题

### 7-1 Hoare 划分的正确性

#### HOARE-PARTITION

In [19]:
def hoare_partition(A, p, r):
    i, j = p - 1, r + 1
    x = A[p]
    while True:
      while True:
        j -= 1
        if A[j] <= x:
          break
      while True:
        i += 1
        if A[i] >= x:
          break
      if i < j:
        A[i], A[j] = A[j], A[i]
      else:
        return j

In [20]:
A = [random.randint(1, 9) for i in range(10)]
print(A)
print(hoare_partition(A, 0, len(A)-1))
print(A)

[7, 2, 5, 6, 4, 1, 3, 7, 9, 7]
7
[7, 2, 5, 6, 4, 1, 3, 7, 9, 7]


#### QUICKSORT-WITH-HOARE-PARTITION  

In [21]:
def quicksort_with_hoare_partition(A, p, r):
  if p < r:
    x = A[p]
    j = hoare_partition(A, p, r)
    # print(A[p:r+1])
    # hoare_partiton 无法保证 j 位置是主元的位置，需要找到主元所在的位置并与j交换
    # 交换的过程中，不能破坏 A[p, j] 中的元素，总是小于 A[j+1, q] 中元素的这一性质
    for i in range(p, r+1):
      if A[i] == x:
        if i > j: # 主元在第二个数组中，则与第二个数组的第一个元素交换
          j = j + 1
        A[i], A[j] = A[j], A[i]
        break
    quicksort_with_hoare_partition(A, p, j-1)
    quicksort_with_hoare_partition(A, j+1, r)

In [22]:
A = [random.randint(1, 10) for i in range(10)]
print(A)
quicksort_with_hoare_partition(A, 0, len(A)-1)
print(A)

[3, 7, 1, 2, 2, 4, 6, 9, 2, 8]
[1, 2, 2, 2, 3, 4, 6, 7, 8, 9]


### 7-2 针对相同元素值的快速排序

#### PARTITION '

1. 实际运行时，借助 $q, t, j$ 三个变量将区间 $[p ,r]$ 划分为三个区间，满足下述关系式：
$$\left\{ \matrix{
  当p \le k \le q \quad A[k] < key\hfill \cr 
  当 q < k \le t \quad A[k]=key \hfill \cr 
  当 t < k \le j \quad A[k]>key\hfill \cr}  \right.$$  
  - 其中左右区间可能为空区间
2. 每次循环当 $j$ 增长时，如果 $A[j] < key$, 则 $q, t$均需要增加 $1$, 且为维持 1 中的循环不变式，需要将 
  - $A[j]$ 的值赋给 $A[q]$
  - $A[t]$ 的值赋给 $A[j]$
  - $A[q]$ 的值赋给 $A[t]$
  - 相当于三个数，两两交换位置。但是由于 $t, j$的值可能相同，必须先运行 $ A[t]\leftrightarrow A[j]$，再运行 $ A[t] \leftrightarrow A[q]$，如果不按此顺序两两交换元素，则当 $t=j$ 时，相当于没有进行交换操作


In [23]:
def partition_handle_same_values(A, p, r):
  q = p - 1
  t = p
  key = A[r]
  A[p], A[r] = A[r], A[p]
  for j in range(p+1, r+1):
    if A[j] < key: 
      q += 1
      t += 1
      A[t], A[j] = A[j], A[t]
      A[t], A[q] = A[q], A[t]
    elif A[j] == key:
      t += 1
      A[j], A[t] = A[t], A[j]
  return (q, t)

In [24]:
A = [random.randint(1,5) for i in range(20)]
print("A is {}, the key value is {}".format(A, A[-1]))
q, t = partition_handle_same_values(A, 0, len(A)-1)
print("{}, {}, {}".format(A[0:(q+1)], A[(q+1):(t+1)], A[(t+1):len(A)]))

A is [5, 2, 2, 5, 1, 1, 1, 3, 1, 2, 2, 3, 4, 3, 4, 5, 1, 2, 1, 4], the key value is 4
[2, 2, 1, 1, 1, 3, 1, 2, 2, 3, 3, 1, 2, 1], [4, 4, 4], [5, 5, 5]


#### RANDOMIZED-PARTITION'

In [25]:
def randomized_partition_handle_same_values(A, p, r):
  i = random.randint(p, r)
  A[i], A[r] = A[r], A[i]
  return partition_handle_same_values(A, p, r)

In [26]:
A = [random.randint(1,5) for i in range(20)]
print("A is {}.".format(A))
q, t = randomized_partition_handle_same_values(A, 0, len(A)-1)
print("{}, {}, {}".format(A[0:(q+1)], A[(q+1):(t+1)], A[(t+1):len(A)]))

A is [2, 5, 3, 2, 4, 1, 1, 5, 1, 2, 1, 2, 2, 2, 2, 2, 3, 4, 1, 2].
[], [1, 1, 1, 1, 1], [5, 3, 5, 2, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 2]


#### RANDOMIZED-QUICKSORT'

In [27]:
def randomized_quicksort_handle_same_values(A, p, r):
  if p < r:
    q, t = randomized_partition_handle_same_values(A, p, r)
    randomized_quicksort_handle_same_values(A, p, q)
    randomized_quicksort_handle_same_values(A, t+1, r)

In [28]:
A = [random.randint(1,5) for i in range(20)]
print("A is {}.".format(A))
randomized_quicksort_handle_same_values(A, 0, len(A)-1)
print("After sorted A is {}".format(A))

A is [4, 5, 4, 1, 3, 4, 3, 2, 2, 5, 3, 2, 5, 2, 4, 1, 5, 3, 1, 5].
After sorted A is [1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5]


In [29]:
A = [random.randint(1, 5) for i in range(1000)]
B = A.copy()
fn_timer(randomized_quicksort)(A, 0, len(A) - 1)
fn_timer(randomized_quicksort_handle_same_values)(A, 0,len(A) - 1)

Total time running randomized_quicksort: 0.078125 s
Total time running randomized_quicksort_handle_same_values: 0.0 s


### 7-4 快速排序的栈深度

#### a.  采用尾递归进行优化

In [31]:
def tail_recursive_quicksort(A, p, r):
  while p < r:
    q = partition(A, p, r)
    tail_recursive_quicksort(A, p, q-1)
    p = q + 1

In [33]:
A = [random.randint(1, 100) for i in range(10)]
print("排序前： A = {}".format(A))
tail_recursive_quicksort(A, 0, len(A) - 1)
print("排序后： A={}".format(A))

排序前： A = [50, 38, 31, 55, 19, 77, 26, 29, 64, 43]
排序后： A=[19, 26, 29, 31, 38, 43, 50, 55, 64, 77]


#### b  
- 当数组已经有序时，栈尝试会为 $\Theta(n)$

#### c  
- 目标是为了避免栈深度过深，所以，可以在递归时，对左右两个子数组中元素较小的数组进行递归

In [35]:
def tail_recursive_quicksort_optimize(A, p, r):
  while p < r:
    q = partition(A, p, r)
    if q < (p + r) // 2:
      tail_recursive_quicksort_optimize(A, p, q-1)
      p = q + 1
    else:
      tail_recursive_quicksort_optimize(A, q+1, r)
      r = q - 1

In [38]:
A = [random.randint(1, 100) for i in range(10)]
print("排序前： A = {}".format(A))
tail_recursive_quicksort_optimize(A, 0, len(A) - 1)
print("排序后： A={}".format(A))

排序前： A = [90, 60, 44, 56, 62, 83, 70, 25, 71, 69]
排序后： A=[25, 44, 56, 60, 62, 69, 70, 71, 83, 90]
